In [1]:
from mesh import Mesh

In [2]:
#minimal
#filename, quality_index = "Data/simplicial_donut_3.ply", 2
#small
#filename, quality_index = "Data/simplicial_donut_3_deform.ply", 2
#filename, quality_index = "Data/31_r1.00_n4_v256.volume.ply", 3
#filename, quality_index = "Data/31_mean_curvature.ply", 7
#medium
filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh.ply", 3
#filename, quality_index = "Data/HS_0109_HeiCuBeDa_GigaMesh_mean_curvature.ply", 7
#large:
#filename, quality_index = "Data/HS_190_HeiCuBeDa_GigaMesh.ply", 3

# Mesh class:
The Mesh class can be initialized using **Mesh()** . It has the following variables and functions:

## Variables:
- **Mesh.filename** filename as string. saved without .ply, but with folder structure

- **Mesh.Vertices** Vertices stored as a dictionary of Vertex objects, keys are Vertex indices

- **Mesh.Edges** Edges stored as a dictionary of Edge objects, keys are numbered

- **Mesh.Faces** Faces stored as a dictionary of Face objects, keys are numbered

- **Mesh.MorseComplex** if not calculated yet: None, otherwise a MorseComplex object with MorseComplex.persistence = 0

- **Mesh.reducedMorseComplexes** a dictionary of MorseComplex objects, that have been reduced by a persistence parameter. The persistence parameter also defines the key of each MorseComplex object

- **Mesh.MorseCells** a dictionary of MorseCells dictionaries. the key for each MorseCell dictionary is given by the persistence of the MorseComplex, these Cells were taken from. Each MorseCell dictionary consits of key, value pairs, where the key gives the label of a cell and the value is a set of vertices that make up this cell.

## Functions:
- **Mesh.load_mesh_ply(filename, quality_index)** loads a .ply file into the Mesh class and takes the value given at quality_index position as the scalar function on the vertices

- **Mesh.info()** prints out Mesh info

- **Mesh.ProcessLowerStars()** calculates the combinatorial gradient and critical simplices; **required for ExtractMorseComplex**

- **Mesh.only_return_ExtractMorseComplex** return MorseComplex without storing it in the Mesh class

- **Mesh.ExtractMorseComplex** calculate MorseComplex and store it in the Mesh class

- **Mesh.only_return_ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, but dont store it in the Mesh class

- **Mesh.ReducedMorseComplex(persistence)** reduce the MorseComplex based on the given persistence parameter, and store it in the Mesh class

- **Mesh.plot_MorseComplex(MorseComplex, filename, path_color=[255,0,255])** plots the critical simplices of a MorseComplex in red (vertex), green (edge) and blue (face) and the according paths connecting them in magenta. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

- **Mesh.ExtractMorseCells(MorseComplex)** takes a MorseComplex and gets all connected cells individually that are enclosed by the lines between critical simplices

- **Mesh.plot_MorseCells(persistence, filename)** plots the connected components in each Cell in the same color. Only writes a ply file containing colored points, so should be used as an overlay for the original mesh

In [3]:
Mesh = Mesh()
Mesh.load_mesh_ply(filename, quality_index, inverted=False)
Mesh.info()
Mesh.ProcessLowerStars()
Mesh.ExtractMorseComplex()
Mesh.MorseComplex.info()

Time read data file: 5.0098378
Time read and prepare data: 28.328500400000003
+-------------------------------------------------------
| Mesh Info
+-------------------------------------------------------
| Filename:  Data/HS_0109_HeiCuBeDa_GigaMesh
| Morse function values range:  [-0.41149557, 0.910333]
+-------------------------------------------------------
| Number of Vertices:  559767
| Number of Edges:  1679295
| Number of Faces:  1119530
+-------------------------------------------------------
| Euler characteristic:  2
+-------------------------------------------------------
Time ProcessLowerStar: 14.760850099999999
Time ExtractMorseComplex and Separatrices: 8.416868200000003
+-------------------------------------------------------
| MorseComplex Info
+-------------------------------------------------------
| Filename:  Data/HS_0109_HeiCuBeDa_GigaMesh
| Persistence of this Complex:  0
+-------------------------------------------------------
| Number of Vertices:  29135
| Number 

In [4]:
red_MSC = Mesh.ReduceMorseComplex(2)

Time cancel critical points with  2  persistence:  7.540399899999997
Persistence was higher than the range of function values,
therefore this complex is maximally reduced and can be used for salient edge extraction.


In [5]:
#Mesh.plot_salient_edge_histogram(100, log=True)

In [6]:
#red_MSC.Separatrices.sort(key=lambda x: x[0])
#for pers,sep in red_MSC.Separatrices:
#    print(pers)


In [7]:
#thr = [1,2,5,8,10,15,20,25,100,600,900]
#min_thresh = 10
#max_thresh = 7
#for thresh in thr:
#    Mesh.plot_improved_salient_edge(str(red_MSC.filename)+"_improved_salient_edge_justSum"+str(thresh), 
#                                    thresh, min_thresh, max_thresh)
#    Mesh.plot_salient_edge(str(red_MSC.filename)+"_salient_edge_justSum"+str(thresh), thresh)

In [8]:
# Calculate reduced Complexes for different persistence levels:

persistences = [0.1]
for thresh in persistences:
    Mesh.ReduceMorseComplex(thresh)
    
# plot MorseComplexes, calculate MorseCells and plot MorseCells
for persistence in persistences:
    reduced_complex = Mesh.reducedMorseComplexes[persistence]
    #reduced_complex.info()
    #Mesh.plot_MorseComplex(reduced_complex, str(reduced_complex.filename)+"_mean_inverted"+str(persistence))
    #Mesh.plot_MorseComplex_pline(persistence, str(Mesh.filename)+"_test_pline_"+str(persistence))
    Mesh.plot_MorseComplex_thresholded_pline(persistence, str(Mesh.filename)+"_test_thresh_pline_"+str(persistence), minimum_length=7, thresh=0.1)
    Mesh.ExtractMorseCells(reduced_complex)
    #Mesh.write_MorseCellLabels(persistence, str(reduced_complex.filename)+"_error_removed_test2_no_bd"+str(persistence))
    #Mesh.plot_MorseCells(persistence, str(reduced_complex.filename)+"_error_removed_test_no_bd"+str(persistence))


Time cancel critical points with  0.1  persistence:  10.275324199999993
Time writing simplified pline file: 1.168288700000005
Time get MorseCells:  6.0237961999999925


In [9]:
'''
graph_pers = 0.1
graph = Mesh.GetConnectivityGraph(graph_pers)
graph.remove_small_components(Mesh, persistence=graph_pers, size_thresh=30)
Mesh.write_MorseCellLabels(graph_pers, str(Mesh.filename)+"_error_removed_test2_smallCompRemoved")
graph.remove_weak_edges(Mesh, persistence=graph_pers, absolute_threshold=0.1)
Mesh.write_MorseCellLabels(graph_pers, str(Mesh.filename)+"_error_removed_test2_weakEdgesRemoved")
'''

'\ngraph_pers = 0.1\ngraph = Mesh.GetConnectivityGraph(graph_pers)\ngraph.remove_small_components(Mesh, persistence=graph_pers, size_thresh=30)\nMesh.write_MorseCellLabels(graph_pers, str(Mesh.filename)+"_error_removed_test2_smallCompRemoved")\ngraph.remove_weak_edges(Mesh, persistence=graph_pers, absolute_threshold=0.1)\nMesh.write_MorseCellLabels(graph_pers, str(Mesh.filename)+"_error_removed_test2_weakEdgesRemoved")\n'